In this notebook, we will get an overview of how to generate word vectors using the various word embedding methods discussed in the lecture

### Objectives:
- Implement Count Vectors with sklearn
- Implement TF-IDF Vectors with sklearn and gensim
- Train and save word2vec model with gensim
- Load Google's pretrained word2vec model
- Load Stanford's pretrained GloVe model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

In [ ]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

corpus = twenty_train.data[0:50]

In [ ]:
print(f"{corpus[0]}")

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

From the result above, we can see that the second dimension gives us the size of our vocabulary.

But why restrict ourselves to single words? We can pass an additional argument to the CountVectorizer() object to add n-grams to our vocabulary.

What is an n-gram?  It's just a collection of n consecutive words. For example:
"New", "York", "City", "subway" are all unigrams
"New York", "York City", "City subway" are bigrams
"New York City", "York City subway" are trigrams
"New York City subway" is a 4-gram

We can specify to include n-grams with the ngram_range argument.  This takes a tuple which specifies the range of n-grams that we should include (inclusively).

In [ ]:
# Include unigrams, bigrams, and trigrams

vectorizer = CountVectorizer(ngram_range=(1,3))
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

A common preprocessing step in many NLP applications is stop-word removal.
Common words like "a", "the", "and" often add a lot of noise, and don't typiccally contribute much to the task we are trying to solve.

CountVectorizer also comes equipped with a way of dealing with common English stop words!

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english')
X = vectorizer.fit_transform(corpus)

print(f"Dimensions of Document-term matrix: {X.toarray().shape}")

# TF-IDF Vectors

Here we will demonstrate two ways to generate TF-IDF vectors with both sklearn and gensim.  It's good to be aware of both methods because depending on your specific workflow, one method might be easier than the other!

In [ ]:
# For sklearn, it's VERY similar to how we did CountVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words='english')
X_tfidf = vectorizer.fit_transform(corpus)

In [ ]:
# How do these two representations compare?
# Let's look at the first 50 dimensions of the first document to gain some intuition

np.set_printoptions(precision=3) # This just makes things a little easier to read

print(f"CountVector: {X.toarray()[0,0:50]}\n\n")

print(f"TFIDF: {X_tfidf.toarray()[0,0:50]}")

Scikit-learn isn't our only option for doing TF-IDF.  Gensim is another popular library for many NLP tasks

In [ ]:
import gensim

In [ ]:
# Tokenize the documents
tokenized_docs = [gensim.utils.simple_preprocess(d) for d in corpus]

# Create a Gensim Dictionary.  This creates an id to word mapping for everything in our vocbulary
# It is NOT the same as the dictionary object in the Python standard library
mydict = gensim.corpora.Dictionary()

# Create a Gensim Corpus object.  This creates a list of tuples for each document.
# The first element of the tuple is the word id, the second is the number of counts
mycorpus = [mydict.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

In [ ]:
# This creates the doc-term matrix as a numpy array.
# Typically these matrices are HUGE so, it's usuall not a great idea to create the full dense doc-term matrix.
# We do it here to illustrate that you can get the same info as we obtained in scikit-learn!
doc_term_matrix = gensim.matutils.corpus2dense(mycorpus, num_terms=len(mydict))

In [ ]:
doc_term_matrix

In [ ]:
# Creating a tf-idf model is very simple!
tfidf = gensim.models.TfidfModel(mycorpus)
tfidf_matrix = gensim.matutils.corpus2dense(tfidf[mycorpus], num_terms=len(mydict))

In [ ]:
tfidf_matrix

# Word2Vec and GloVe

Word2Vec is a very powerful and useful word embedding method.  The math can get a little sticky, but luckily Gensim comes equipped with ways for us to train our own Word2Vec model, or load in a pre-trained word2vec model.  Let's check it out!

In [ ]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

tokenized_docs = [gensim.utils.simple_preprocess(d) for d in documents]


In [ ]:
# size refers to the desired dimension of our word vectors
# window refers to the size of our context window
# sg means that we are using the Skip-gram architecture

model = gensim.models.Word2Vec(tokenized_docs, size=10, window=2,min_count=1, sg=1)

In [ ]:
print(model['human'])

Training our own model with word2vec is pretty cool, but it requires us to have a large corpus of data.

Fortunately, research groups at Stanford and Google have made their pre-trained word embeddings publicly available for us to use!

Google's word2vec: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

GloVe:  https://nlp.stanford.edu/projects/glove/

Just note that these model's will require ~4 GB of RAM to fit in memory

In [ ]:
# Path to where the word2vec file lives
google_vec_file = '/Users/adam/Downloads/GoogleNews-vectors-negative300.bin'

In [ ]:
# Load it!  This might take a few minutes...
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [ ]:
# We can access individual word vectors using a dictionary-like syntax
model['Metis']

In [ ]:
# Some cool results!

model.most_similar('meeting' ,topn=8)

In [ ]:
model.most_similar('president' ,topn=5)

In [ ]:
# Here's an analogy task!

model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
model.most_similar(positive=['Paris', 'England'], negative=['London'])

Using GloVe with Gensim requires a little extra leg work, but it's not too bad.
The problem is that the file format that is publicly available doesn't play nice with Gensim.
Luckily, Gensim provides a handy method of converting it!

In [ ]:
glove_file = glove_dir = '/Users/adam/Downloads/glove.6B/glove.6B.100d.txt'

w2v_output_file = '/Users/adam/Downloads/glove.6B/glove.6B.100d.txt.w2v'

# The following utility converts file formats
gensim.scripts.glove2word2vec.glove2word2vec(glove_file, w2v_output_file)

In [ ]:
# Now we can load it!
model = gensim.models.KeyedVectors.load_word2vec_format(w2v_output_file, binary=False)

In [ ]:
# How does it compare to the previous examples we did with word2vec?
model.most_similar('meeting' ,topn=8)

In [ ]:
model.most_similar('president' ,topn=5)

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

# Your Turn!

- Using either word2vec or GloVe, what interesting analogies or relationships?

- Given a short piece of text (like a tweet) what strategies can you think of to create a "tweet vector"?